In [ ]:
#importação de pacotes and utilitarios

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import pingouin as pg
import plotly.express as px 
import plotly.io as pio
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from scipy.stats import chi2_contingency
import statsmodels.api as sm
import plotly.graph_objects as go
pio.renderers.default='browser'

def convertToNumber(banco, coluna):
    banco[coluna] = pd.to_numeric(banco[coluna], errors='coerce')

In [ ]:
# importação de banco de dados
dados_pisa = pd.read_csv('./bancos_dados/notas_pisa.csv', delimiter=',')
dados_pisa.info()

In [ ]:
"""
 Em um primeiro momento só iremos utilizar as notas de 2022, 
 por este motivo será gerado outro banco de dados sem as notas de 2018
 E faremos o tratamento dos dados
"""

#remove as colunas que não usaremos
if 'mathematics_2018' in dados_pisa.columns:
    dados_pisa.drop(columns=['mathematics_2018', 'reading_2018', 'science_2018'], inplace=True)

#realizarmos padronização dos dados, pois as notas estão em formato texto, iremos transformar para numeros
convertToNumber(dados_pisa, 'mathematics_2022')
convertToNumber(dados_pisa, 'reading_2022')
convertToNumber(dados_pisa, 'science_2022')

#remove as linhas NAN
dados_pisa.dropna(inplace=True)
dados_pisa.info()

In [ ]:
# iremos iniciar com a analise PCA

"""para gerar o mapa de calor a matriz deve possuir apenas dados categoricos, por isso iremos remover
as varveis COUNTRY E GROUP, porem agora iremos gerar outro dataFrame para manter o original e
depois fazer um merge """

pisa_pca = dados_pisa.drop(columns=['country', 'group'])


In [ ]:
# Geração de mapa de calor para verificar se existe relação forte entre as variaveis
# Esta mapa é gerado apartir de uma matriz de correção, esta matriz é gerada pelo comando pisa_pca.corr()
# INTERPRETAÇÃO - no mapa de calor, quanto mais perto de 1, maior é a relação entre as variaveis
sns.heatmap(pisa_pca.corr(), annot=True, 
            cmap = plt.cm.Purples,
            annot_kws={'size':7})